In [1]:
import sys
import unittest
import functools

import numpy as np
from scipy.linalg import expm
from scipy.special import erf

from qiskit.providers.aer.backends import PulseSimulator

from qiskit.compiler import assemble
from qiskit.quantum_info import state_fidelity
from qiskit import pulse
from qiskit.pulse import (Schedule, Play, ShiftPhase, SetPhase, Delay, Acquire, SamplePulse,
                          DriveChannel, ControlChannel, AcquireChannel, MemorySlot)
from qiskit.providers.aer.pulse.de.DE_Methods import ScipyODE
from qiskit.providers.aer.pulse.de.DE_Options import DE_Options
from qiskit.providers.aer.pulse.system_models.pulse_system_model import PulseSystemModel
from qiskit.providers.aer.pulse.system_models.hamiltonian_model import HamiltonianModel
from qiskit.providers.models.backendconfiguration import UchannelLO

In [7]:
def _1Q_gauss_sched(amp=1., sig=20, duration=140):
    """Creates a runnable schedule for 1Q with a constant drive pulse of a given length.

    Args:
        total_samples (int): length of pulse
        amp (float): amplitude of constant pulse (can be complex)

    Returns:
        schedule (pulse schedule): schedule with a drive pulse followed by an acquire
    """

    # set up constant pulse for doing a pi pulse
    drive_pulse = pulse.Gaussian(duration=duration, amp=amp, sigma=sig)
    schedule = Schedule()
    schedule |= Play(drive_pulse, DriveChannel(0))
    schedule |= Acquire(1, AcquireChannel(0), MemorySlot(0)) << schedule.duration

    return schedule

def _system_model_1Q(omega_0, r):
    """Constructs a standard model for a 1 qubit system.

    Args:
        omega_0 (float): qubit frequency
        r (float): drive strength

    Returns:
        PulseSystemModel: model for qubit system
    """

    hamiltonian = {}
    hamiltonian['h_str'] = ['2*np.pi*omega0*0.5*Z0', '2*np.pi*r*0.5*X0||D0']
    hamiltonian['vars'] = {'omega0': omega_0, 'r': r}
    hamiltonian['qub'] = {'0': 2}
    ham_model = HamiltonianModel.from_dict(hamiltonian)

    u_channel_lo = []
    subsystem_list = [0]
    dt = 1.

    return PulseSystemModel(hamiltonian=ham_model,
                            u_channel_lo=u_channel_lo,
                            subsystem_list=subsystem_list,
                            dt=dt)

backend_sim = PulseSimulator()

In [11]:
# qubit frequency and drive frequency
omega_0 = 5.
omega_d = omega_0

# drive strength and length of pulse
r = 0.02

system_model = _system_model_1Q(omega_0, r)

# set up constant pulse for doing a pi pulse
schedule = _1Q_gauss_sched()

# set up schedule and qobj
qobj = assemble([schedule],
                backend=backend_sim,
                meas_level=2,
                meas_return='single',
                meas_map=[[0]],
                qubit_lo_freq=[omega_d],
                memory_slots=1,
                shots=1)

# set backend backend_options including initial state
y0 = np.array([0.0, 1.0])
backend_options = {'initial_state' : y0, 'solver_options': {'atol': 1e-10, 'rtol': 1e-10}}

# run simulation
%time result = backend_sim.run(qobj, system_model=system_model, backend_options=backend_options).result()

1.0847580432891846
CPU times: user 1.08 s, sys: 17.7 ms, total: 1.1 s
Wall time: 1.09 s


In [9]:
result.results[0].__dict__

{'_metadata': {'seed_simulator': 1963405537,
  'time_taken': 0.33777809143066406},
 'shots': 1,
 'success': True,
 'data': <qiskit.result.models.ExperimentResultData at 0x7fe170583a30>,
 'meas_level': 2,
 'header': namespace(memory_slots=1, name='sched4', ode_t=140.0),
 'status': 'DONE',
 'meas_return': 'single'}

In [5]:
0.02 * total_samples

NameError: name 'total_samples' is not defined

In [ ]:
help(pulse.Gaussian)